In [2]:
import os
import numpy as np
import pandas as pd
from pprint import pprint
from slugify import slugify


def open_dataset(file_name):
    return pd.read_csv(filepath_or_buffer=file_name, delimiter="\t", encoding="utf-8", header=0)


itineraries = open_dataset('../../data_sncf/timetables.csv')
itineraries = itineraries.sort_values(
    by=['trip_id', 'trajet', 'duree'], ascending=[True, True, True])

# dataframe refacto
trajet_splited = itineraries['trajet'].str.split(' - ')

for key, values in trajet_splited.iteritems():
    trajet_splited[key] = [slugify(x) for x in trajet_splited[key]]

itineraries['start'] = trajet_splited.str[0]
itineraries['end'] = trajet_splited.str[1]

itineraries.drop('trajet', axis=1, inplace=True)
itineraries.drop('trip_id', axis=1, inplace=True)

# inverse start & end
itinerariesInversed = itineraries.rename(columns={'start': 'end', 'end': 'start'})
itineraries = pd.concat([itineraries, itinerariesInversed])

# remove duplicate on start/end
itineraries = itineraries.drop_duplicates(['start','end'], keep= 'last')
itineraries.head()



,duree,start,end
0,138,gare-de-le-havre,gare-de-paris-st-lazare
1,145,gare-de-dieppe,gare-de-paris-st-lazare
2,97,gare-de-paris-st-lazare,gare-de-rouen-rive-droite
3,194,gare-de-cherbourg,gare-de-paris-st-lazare
4,149,gare-de-caen,gare-de-paris-st-lazare


In [3]:
# Filter all possible localisations
locsStart = itineraries['start'].tolist()
locsEnd = itineraries['end'].tolist()
locs = list(dict.fromkeys(locsStart + locsEnd))
# len(locs)
locs


['gare-de-le-havre',
 'gare-de-dieppe',
 'gare-de-paris-st-lazare',
 'gare-de-cherbourg',
 'gare-de-caen',
 'gare-de-granville',
 'gare-de-dreux',
 'gare-de-bourges',
 'gare-de-paris-austerlitz',
 'gare-de-nevers',
 'gare-de-argentan',
 'gare-de-orleans',
 'gare-de-aubrais-les',
 'gare-de-paris-bercy',
 'gare-de-calais-ville',
 'gare-de-boulogne-ville',
 'gare-de-amiens',
 'gare-de-cambrai-ville',
 'gare-de-maubeuge',
 'gare-de-paris-gare-du-nord',
 'gare-de-compiegne',
 'gare-de-le-mans',
 'gare-de-nogent-le-rotrou',
 'gare-de-chartres',
 'gare-de-blois-chambord',
 'gare-de-lyon-perrache',
 'gare-de-latour-de-carol-enveitg',
 'gare-de-font-romeu-odeillo-via',
 'gare-de-briancon',
 'gare-de-romans-bourg-de-peage',
 'gare-de-gap',
 'gare-de-aix-en-provence',
 'gare-de-marseille-st-charles',
 'gare-de-les-arcs-draguignan',
 'gare-de-grenoble',
 'gare-de-chambery-chal-les-eaux',
 'gare-de-annecy',
 'gare-de-avignon-centre',
 'gare-de-lyon-part-dieu',
 'gare-de-dijon-ville',
 'gare-de-maco

In [4]:
# get all possible itineraries from a localisation
def get_itineraries(fromLoc):
    return itineraries.loc[itineraries['start'] == fromLoc]


In [5]:
# update distance of localisation from source
def updateDistanceFromSource(matrix, itineraries):
    # print(itineraries)

    for index, row in itineraries.iterrows():

        # if initerary is not yet processed
        if matrix[row['end']]['processed'] == False:

            oldValue = matrix[row['end']]['distanceFromSource']
            newValue = matrix[row['start']
                              ]['distanceFromSource'] + row['duree']

            if oldValue == 'infinite':
                matrix[row['end']]['distanceFromSource'] = newValue
                matrix[row['end']]['fromLoc'] = row['start']
            else:
                if newValue < oldValue:
                    matrix[row['end']]['distanceFromSource'] = newValue
                    matrix[row['end']]['fromLoc'] = row['start']

    # print("-----------------------------------------------------------------")
    # print("---------- MISE A JOUR DES DISTANCES DEPUIS LA SOURCE -----------")
    # print("-----------------------------------------------------------------")
    # print(" ")

    return matrix


In [6]:
# Find the minimum distance that has not been processed and that is not infinite
def findMiniDistanceFromSource(matrix):
    
    toProcess = {}
    
    for index, row in matrix.items():
        if row['processed'] == False and row['distanceFromSource'] != 'infinite':
            toProcess[index] = row
            
    # print("-------------------- Les itineraires pas encore traités ---------------------")
    # print(" ")
    # [print(key,':',value) for key, value in toProcess.items()]
    # print(" ")
    
    miniValue = 9999999999
    location = ""
    
    for index, row in toProcess.items():
    
        if row['distanceFromSource'] < miniValue:
            miniValue = row['distanceFromSource']
            location = index
            
       
    # if location != "":    
    #     print("-------------------- La distance la plus courte pas encore traitée ---------------------")
    #     print(" ")
    #     print("Etape : ", location, " - Distance depuis la source : ", miniValue)
    #     print(" ")
    # else:
    #     print("------------------------------------------------------------------------")
    #     print("-------------------- Plus de trajets lié à traiter ---------------------")
    #     print("------------------------------------------------------------------------")
    #     print(" ")
        
    return location
    
                
    

In [7]:
# find the shortest distance between source and destination
def get_best_itinerary(matrix, source, destination):
    toProcess = {}
    distance = ""
    fromLoc = ""
    itineraries = []

    for index, row in matrix.items():
        if row['processed'] == True and row['distanceFromSource'] != 'infinite':
            toProcess[index] = row

    # print(toProcess)
    if destination in toProcess.keys():
        # print(toProcess[destination])
        distance = toProcess[destination]["distanceFromSource"]
        fromLoc = toProcess[destination]["fromLoc"]
        itineraries.insert(0, destination)

        while fromLoc != source:
            itineraries.insert(0, fromLoc)
            fromLoc = toProcess[fromLoc]["fromLoc"]

        itineraries.insert(0, source)

        return {
            'distance': distance,
            'itineraries': itineraries
        }

    else:
        print("-----------------------------------------------------------------")
        print("-------------------- Ce trajet n'existe pas ---------------------")
        print("-----------------------------------------------------------------")
        print("")

        return "None"


In [12]:
# Find the shortest path among all possible sources and destinations 
def dijsktra(df, source, destination):
    matrixLoc = {}
    
    locsStart = list(set(df['start'].tolist()))
    locsEnd = list(set(df['end'].tolist()))
    locs = list(dict.fromkeys(locsStart + locsEnd)) 
    
    # filter possible localisations
    locsStart = list(filter(lambda k: source in k, locs))
    locsEnd = list(filter(lambda k: destination in k, locs))
    
    # print("((((((((((((((aaa))))))))))))))")
    # print(locsStart)
    # print("((((((((((((((aaa))))))))))))))")
    # print(locsEnd)
    # print("((((((((((((((zzz))))))))))))))")
    
    allResultFound = []

    # Try to find a initerary from each train station with name searched
    for locStart in locsStart:
        matrixLoc = {}
        
        for locEnd in locsEnd:
            if locEnd == locStart:
                continue
            
            matrixLoc = {}
    
            # print("Trajet : ", locStart, " - ", locEnd)
            # print(" ")
            
            # initialize matrix
            for loc in locs:
                matrixLoc[loc] = {
                    'distanceFromSource': 'infinite',
                    'fromLoc': '',
                    'processed': False,
                }
                
            # initialize matrix source values
            matrixLoc[locStart]['distanceFromSource'] = 0
            matrixLoc[locStart]['fromLoc'] = locStart
            matrixLoc[locStart]['processed'] = True
            
            nextLocation = locStart
            
            while nextLocation != "":

                if not get_itineraries(nextLocation).empty:
                    matrixLoc = updateDistanceFromSource(matrixLoc, get_itineraries(nextLocation))
                # else:
                    # print("Aucun itinéraire en partant de ", nextLocation)
                    # print(" ")
                    
                nextLocation = findMiniDistanceFromSource(matrixLoc)

                if nextLocation != "":
                    matrixLoc[nextLocation]['processed'] = True
            
            result = get_best_itinerary(matrixLoc, locStart, locEnd)
            
            if result != "None":
                allResultFound.append(result) 


    # pprint(allResultFound)
    
    results = []
    for result in allResultFound:
        if result
        
        
        
        
    return allResultFound
    

In [13]:

def get_itinerary(df=itineraries, locArray=[]):
    locArray = [slugify(x) for x in locArray]

    fullTrip = []
    distance = 0

    while len(locArray) >= 2:
        locStart = locArray[0]
        locEnd = locArray[1]

        # print(locStart)
        # print(locEnd)

        result = dijsktra(df, locStart, locEnd)
        return result

        if result:
            fullTrip = fullTrip + result['itineraries']
            distance = distance + result['distance']
            locArray.pop(0)

        else:
            print("-----------------------------------------------------------------")
            print("---------------------- Aucun trajet trouvé ----------------------")
            print("-----------------------------------------------------------------")
            print("")

            return

    fullTrip = list(dict.fromkeys(fullTrip))

    return {
        'distance': distance,
        'itineraries': fullTrip
    }


In [14]:

# TODO : fix this
# Probleme supposé : les trajet inversés sont possibles dans dijtra directement au lieu de step_machin
# solution : ajouter dans le dataset directement les trajet retour inverse
# get_itinerary(itineraries, ["Paris", "Marseille"])

get_itinerary(itineraries, ["Paris", "Lyon", "Marseille"])
# get_itinerary(itineraries, ["Paris", "Lyon", "Bordeaux"])
# get_itinerary(itineraries, ["Persan-Beaumont", "RENNES"])
# get_itinerary(itineraries, ["Bordeaux", "Paris", "Lille" ])


[{'distance': 467,
  'itineraries': ['gare-de-paris-austerlitz',
                  'gare-de-bourges',
                  'gare-de-nevers',
                  'gare-de-moulins-sur-allier',
                  'gare-de-paray-le-monial',
                  'gare-de-lozanne',
                  'gare-de-lyon-gorge-de-loup']},
 {'distance': 598,
  'itineraries': ['gare-de-paris-austerlitz',
                  'gare-de-bourges',
                  'gare-de-nevers',
                  'gare-de-moulins-sur-allier',
                  'gare-de-paray-le-monial',
                  'gare-de-lozanne',
                  'lyon-st-paul-quai-bondy',
                  'gare-de-sain-bel',
                  'lyon-st-paul-la-feuilee']},
 {'distance': 413,
  'itineraries': ['gare-de-paris-austerlitz',
                  'gare-de-bourges',
                  'gare-de-nevers',
                  'gare-de-lyon-perrache',
                  'gare-de-givors-ville',
                  'gare-de-lyon-part-dieu']},
 {'distance': 2

[{'distance': 467,
  'itineraries': ['gare-de-paris-austerlitz',
   'gare-de-bourges',
   'gare-de-nevers',
   'gare-de-moulins-sur-allier',
   'gare-de-paray-le-monial',
   'gare-de-lozanne',
   'gare-de-lyon-gorge-de-loup']},
 {'distance': 598,
  'itineraries': ['gare-de-paris-austerlitz',
   'gare-de-bourges',
   'gare-de-nevers',
   'gare-de-moulins-sur-allier',
   'gare-de-paray-le-monial',
   'gare-de-lozanne',
   'lyon-st-paul-quai-bondy',
   'gare-de-sain-bel',
   'lyon-st-paul-la-feuilee']},
 {'distance': 413,
  'itineraries': ['gare-de-paris-austerlitz',
   'gare-de-bourges',
   'gare-de-nevers',
   'gare-de-lyon-perrache',
   'gare-de-givors-ville',
   'gare-de-lyon-part-dieu']},
 {'distance': 270,
  'itineraries': ['gare-de-paris-austerlitz',
   'gare-de-vierzon',
   'gare-de-montlucon-ville',
   'urcay-hotel-du-lyon-d-o']},
 {'distance': 580,
  'itineraries': ['gare-de-paris-austerlitz',
   'gare-de-bourges',
   'gare-de-nevers',
   'gare-de-moulins-sur-allier',
   'gare-d